# Performing and analyzing a binding free energy calculation with YANK

In this exercise, we will perform a binding free energy calculation.

When you are done with this exercise, save it under `Chem456-2022F/exercises` on Google Drive. It will be graded as satisfactory or unsatisfactory based on correctly completing the sections after `-->`. Do not remove the symbol `-->`.

# Part 0 - Setting up on Google Colab

### Mounting Google Drive

In [1]:
import os
if not os.path.isdir('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


## Installing packages

The following code cells will install all required packages, if you are working on [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb). Installing the [condacolab](https://github.com/jaimergp/condacolab) package will restart the kernel, which is intended. This notebook can also be used on a local computer but requires considerable computing power.

In [2]:
!pip install munkres gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
try:
    import google.colab
    !pip install condacolab
    import condacolab
    condacolab.install()
except ModuleNotFoundError:
    pass

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:20
🔁 Restarting kernel...


We will install this software stack:
* yank - performs absolute binding free energy calculations
* openmm - molecular mechanics engine for yank
* cudatoolkit - openmm dependency
* matplotlib - for plotting results
* seaborn - improving matplotlib figures
* py3Dmol - visualizing structures
* mdanalysis - for analyzing results, e.g. binding poses

In [1]:
try:
    import condacolab
    from google.colab import files
    from IPython.display import clear_output
    condacolab.check()
    !conda install -q -y -c conda-forge yank openmm cudatoolkit matplotlib seaborn py3Dmol mdanalysis
except ModuleNotFoundError:
    on_colab = False
else:
    #check if installation was succesful
    try:
        import openmm
        on_colab = True
        clear_output()  # clear the excessive installation outputs
        print("Dependencies successfully installed!")
    except ModuleNotFoundError:
        print("Error while installing dependencies!")

Dependencies successfully installed!


# Part 1 - Running YANK

In [2]:
WORKING_DIR = '/content' # If preferred, this could be changed to somewhere on Google Drive
%cd {WORKING_DIR}

receptor = 'BRD'
ligand = 'IR6'

import os

# The system was prepared based on exercise 6
# The PDB file for the receptor was extracted from `complex.pdb`.
# The ligand was converted from sdf to mol2. 
if not os.path.isfile('BRD-IR6-prepared.zip'):
  !gdown 1SUvN1jDwFsyEFAGNH0n6zHI3sP07xbYQ
if not os.path.isdir('input'):
  !unzip BRD-IR6-prepared.zip

/content
Downloading...
From: https://drive.google.com/uc?id=1SUvN1jDwFsyEFAGNH0n6zHI3sP07xbYQ
To: /content/BRD-IR6-prepared.zip
100% 35.9k/35.9k [00:00<00:00, 45.2MB/s]
Archive:  BRD-IR6-prepared.zip
   creating: input/
  inflating: __MACOSX/._input        
  inflating: input/BRD.pdb           
  inflating: __MACOSX/input/._BRD.pdb  
  inflating: input/IR6.mol2          
  inflating: __MACOSX/input/._IR6.mol2  


YANK accepts run options in a file based on a simple format, YAML. The full set of options are described in the [YAML file documentation](http://getyank.org/latest/yamlpages/index.html).

In [3]:
F = open(f'{receptor}-{ligand}-implicit-repex.yaml','w')
F.write(f'''# Set the general options of our simulation
options:
  minimize: yes
  verbose: yes
  default_number_of_iterations: 300
  temperature: 300*kelvin
  pressure: null
  output_dir: {WORKING_DIR}/{receptor}-{ligand}-implicit-repex

# Configure the specific molecules we will use for our systems
# Note: We do not specify what the "receptor" and what the "ligand" is yet
molecules:
  # Define our receptor, we can call it whatever we want so we just use its name here as the directive
  {receptor}:
    filepath: {WORKING_DIR}/input/{receptor}.pdb
    strip_protons: yes
  # Define our ligand molecule
  {ligand}:
    filepath: {WORKING_DIR}/input/{ligand}.mol2
    # Get the partial charges for the ligand by generating them from antechamber with the AM1-BCC charge method
    antechamber:
      charge_method: bcc

# Define the solvent for our system, here we use GBSA Implicit Solvent
solvents:
  # We can title this solvent whatever we want. We just call it "GBSA" for easy remembering
  GBSA:
    nonbonded_method: NoCutoff # Main definition of the nonbonded method
    implicit_solvent: OBC2 # Onufriev-Bashford-Case GBSA model, name is related to Implicit solvents in OpenMM

# Define the systems: What is the ligand, receptor, and solvent we put them in
systems:
  # We can call our system anything we want, this example just uses a short name for the receptor hyphenated with the ligand
  {receptor}-{ligand}:
    # These names all use the names we defined previously
    receptor: {receptor}
    ligand: {ligand}
    solvent: GBSA
    leap:
      parameters: [leaprc.protein.ff14SB, leaprc.gaff2, leaprc.water.tip4pew]

# The protocols define the alchemical path each phase will take, we use the same lambda values, though they could be different
protocols:
  # Call the protocol whatever you would like, here we name it based on the type of calculation we are running
  absolute-binding:
    complex:
      alchemical_path:
        lambda_electrostatics: [1.00, 1.00, 1.00, 1.00, 1.00, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.10, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
        lambda_sterics:        [1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 0.95, 0.90, 0.85, 0.80, 0.75, 0.70, 0.65, 0.60, 0.55, 0.50, 0.45, 0.40, 0.35, 0.30, 0.25, 0.20, 0.15, 0.10, 0.05, 0.02, 0.00]
        lambda_restraints:     [0.00, 0.25, 0.50, 0.75, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00]
    solvent:
      alchemical_path:
        lambda_electrostatics: [1.00, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.10, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
        lambda_sterics:        [1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.10, 0.00]

# Here we combine the system and the protocol to make an experiment
experiments:
  system: {receptor}-{ligand}
  protocol: absolute-binding
  restraint:
    type: Harmonic # Keep the ligand in the original pose
''');
F.close()

# This gave reasonable results
# lambda_electrostatics: [1.00, 1.00, 1.00, 1.00, 1.00, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.10, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
# lambda_sterics:        [1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40, 0.30, 0.25, 0.20, 0.15, 0.10, 0.05, 0.02, 0.00]
# lambda_restraints:     [0.00, 0.25, 0.50, 0.75, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00]

#### --> Are simulations run at constant pressure or volume? Hint: see the [YAML options documentation](http://getyank.org/latest/yamlpages/options.html#yaml-options-head).

#### --> Why does the absolute binding free energy include alchemical paths for "complex" and "solvent"? Hint: see the [thermodynamic cycle](http://getyank.org/latest/theory.html).

#### --> Describe what happens to the electrostatic coupling, steric coupling, and restraints during the alchemical path for the complex thermodynamic process.

The following cell will either run YANK or download previous YANK results. If you run it, you should use a GPU run time. Otherwise the progam will run very slowly. With the GPU, it took about 40 minutes for the complex phase and 10 minutes for the solvent phase.

In [4]:
run_YANK = True

if run_YANK:
  from yank.experiment import ExperimentBuilder
  yaml_builder = ExperimentBuilder(script=f'{receptor}-{ligand}-implicit-repex.yaml')
  yaml_builder.run_experiments()

  # This compresses and stores the results on Google Drive
  !zip -r {receptor}-{ligand}-implicit-repex.zip {receptor}-{ligand}-implicit-repex

  import shutil
  shutil.copy(f'{receptor}-{ligand}-implicit-repex.zip',\
              '/content/drive/MyDrive/Chem456-2022F/exercises/11-yank/')
else:
  import os
  if not os.path.isfile(f'{receptor}-{ligand}-implicit-repex.zip'):
    !gdown 1MQ94vQRguYU_L_KRm8G0rukjZVlsO5D9
    # BRD-IR6-implicit-repex-29.zip
    # https://drive.google.com/file/d/1-0g4kacssaJAOWUn9BlXz4SdSslHMb1Y/view?usp=sharing
    # BRD-IR6-implicit-repex.zip
    # https://drive.google.com/file/d/1MQ94vQRguYU_L_KRm8G0rukjZVlsO5D9/view?usp=sharing
  if not os.path.isdir(f'{receptor}-{ligand}-implicit-repex'):
    !unzip {receptor}-{ligand}-implicit-repex.zip

2022-11-14 16:03:09,984: Setting CUDA platform to use precision model 'mixed'.
2022-11-14 16:03:10,145: Single node: executing <function ExperimentBuilder._check_resume at 0x7f0e14036680>
2022-11-14 16:03:10,150: Running _setup_molecules serially.
2022-11-14 16:04:42,773: Fixing net charge from 0.003998999999998948 to 8.326672684688674e-17
2022-11-14 16:04:42,781: Running get_system serially.
2022-11-14 16:04:42,783: Setting up the systems for BRD and IR6 using solvent GBSA
2022-11-14 16:04:42,785: Setting up solvent phase
2022-11-14 16:04:42,937: Setting up complex phase
2022-11-14 16:04:43,335: Single node: executing <function ExperimentBuilder._generate_yaml at 0x7f0e14036d40>
2022-11-14 16:04:43,347: Running _generate_experiment_protocol serially.
2022-11-14 16:04:43,350: DSL string for the ligand: "resname UNL"
2022-11-14 16:04:43,352: DSL string for the solvent: "auto"
2022-11-14 16:04:43,356: Reading phase complex
2022-11-14 16:04:43,358: prmtop: /content/BRD-IR6-implicit-repex/

Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

Streaming output truncated to the last 5000 lines.
2022-11-14 16:42:39,611: LangevinSplittingDynamicsMove: Integrator using CUDA platform.
2022-11-14 16:42:39,863: Running _get_replica_move_statistics serially.
2022-11-14 16:42:39,865: Propagating all replicas took    9.327s
2022-11-14 16:42:39,872: Running _compute_replica_energies serially.
2022-11-14 16:42:40,484: Computing energy matrix took    0.612s
2022-11-14 16:42:40,485: Single node: executing <function MultiStateSampler._report_iteration at 0x7f0e1fdd9a70>
2022-11-14 16:42:40,494: Single node: executing <function MultiStateSampler._report_iteration_items at 0x7f0e1fdd9d40>
2022-11-14 16:42:40,499: Iteration 221 not on the Checkpoint Interval of 50. Sampler State not written.
2022-11-14 16:42:40,542: Storing sampler states took    0.043s
2022-11-14 16:42:40,643: Writing iteration information to storage took    0.149s
2022-11-14 16:42:40,645: Single node: executing <function MultiStateSampler._online_analysis at 0x7f0e1fddc8c0>

Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

Streaming output truncated to the last 5000 lines.
2022-11-14 17:04:00,524: ********************************************************************************
2022-11-14 17:04:00,525: Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x7f0e19955560>
2022-11-14 17:04:00,527: Mixing replicas...
2022-11-14 17:04:00,530: Mixing of replicas took    0.001s
2022-11-14 17:04:00,531: Accepted 11142/18522 attempted swaps (60.2%)
2022-11-14 17:04:00,534: Propagating all replicas...
2022-11-14 17:04:00,535: Running _propagate_replica serially.
2022-11-14 17:04:00,537: LangevinSplittingDynamicsMove: Integrator using CUDA platform.
2022-11-14 17:04:00,636: LangevinSplittingDynamicsMove: Integrator using CUDA platform.
2022-11-14 17:04:00,731: LangevinSplittingDynamicsMove: Integrator using CUDA platform.
2022-11-14 17:04:00,828: LangevinSplittingDynamicsMove: Integrator using CUDA platform.
2022-11-14 17:04:00,923: LangevinSplittingDynamicsMove: Integrator using CUDA platform.
2

  adding: BRD-IR6-implicit-repex/ (stored 0%)
  adding: BRD-IR6-implicit-repex/experiments/ (stored 0%)
  adding: BRD-IR6-implicit-repex/experiments/complex_checkpoint.nc (deflated 1%)
  adding: BRD-IR6-implicit-repex/experiments/complex.nc (deflated 1%)
  adding: BRD-IR6-implicit-repex/experiments/solvent.nc (deflated 22%)
  adding: BRD-IR6-implicit-repex/experiments/solvent_checkpoint.nc (deflated 20%)
  adding: BRD-IR6-implicit-repex/experiments/complex_real_time_analysis.yaml (deflated 73%)
  adding: BRD-IR6-implicit-repex/experiments/solvent_real_time_analysis.yaml (deflated 74%)
  adding: BRD-IR6-implicit-repex/experiments/analysis.yaml (deflated 11%)
  adding: BRD-IR6-implicit-repex/experiments/experiments.yaml (deflated 71%)
  adding: BRD-IR6-implicit-repex/experiments/experiments.log (deflated 93%)
  adding: BRD-IR6-implicit-repex/setup/ (stored 0%)
  adding: BRD-IR6-implicit-repex/setup/systems/ (stored 0%)
  adding: BRD-IR6-implicit-repex/setup/systems/BRD-IR6/ (stored 0%)
 

# Part 2 - Simulation Health Report

The code below is taken from a [template health report](https://github.com/choderalab/yank/blob/master/Yank/reports/YANK_Health_Report_Template.ipynb) within YANK. It will help us understand whether the simulation is converged.

## General Settings
========

Mandatory Settings
----------------
* `store_directory`: Location where the experiment was run. This has an `analysis.yaml` file and two `.nc` files.

Optional Settings
----------------
* `decorrelation_threshold`: When number of decorrelated samples is less than this percent of the total number of samples, raise a warning. Default: `0.1`.
* `mixing_cutoff`: Minimal level of mixing percent from state `i` to `j` that will be plotted. Default: `0.05`.
* `mixing_warning_threshold`: Level of mixing where transition from state `i` to `j` generates a warning based on percent of total swaps. Default: `0.90`.
* `phase_stacked_replica_plots`: Boolean to set if the two phases' replica mixing plots should be stacked one on top of the other or side by side. If `True`, every replica will span the whole notebook, but the notebook will be longer. If `False`, the two phases' plots will be next to each other for a shorter notebook, but a more compressed view. Default `False`.

In [ ]:
# Mandatory Settings
store_directory = f'{WORKING_DIR}/{receptor}-{ligand}-implicit-repex/experiments'
# analyzer_kwargs = ANALYZERKWARGSBLANK

# Optional Settings
decorrelation_threshold = 0.1
mixing_cutoff = 0.05
mixing_warning_threshold = 0.90
phase_stacked_replica_plots = False

### Data Imports

These are the imports and files which will be referenced for the report

In [ ]:
from matplotlib import pyplot as plt
from yank.reports import notebook
%matplotlib inline
report = notebook.HealthReportData(store_directory)
report.report_version()

### General Simulation Data

Reports the number of iterations, states, and atoms in each phase. If no checkpoint file is found, the number of atoms is reported as `No Cpt.` as this information is inferred from the checkpoint file. All other information comes from the analysis file.

In [ ]:
report.general_simulation_data()

## Equilibration
=============

How to interpret these plots
--------------------------
Shown is the potential energy added up across all replicas (black dots), the moving average (red line), and where we have auto-detected the equilibration (blue line) for each phase. Finally, the total number of decorrelated samples for each phase is attached to each plot.

You want to see a majority of samples to the right of the blue line and the red line converging to a constant value.  If you do not see these trends or you think there are insufficient samples, please consider running for longer. 

For additional information on the theory of these charts, please see the Equilibration Primer at the Appendix of the report

See Something Odd?
-----------------

* **The scatter plot Y scale looks large and the equilibrium line is at Iteration 0**

This normally happens when the energy from index 0 comes from the energy minimized configuration. Because this configuration is technically not from the equilibrium distribution, it can have large energies far from the true mean equilibrium energy. This can cause the ``detectEquilibration`` algorithm we use to think that the jump in energy from the minimized to the equilibrated is the scale of the energy fluctuations, and therefore all other fluctuations appear as though they are equilibrated. Look close at the first few points: is there are there a few points which are a large shift on the first few steps? If so, consider removing those first few points from the timeseries.

*Solution:* Increase ``discard_from_start``

*Warning:* Some simulations (frequently solvent simulations) are often equilibrated *starting* at iteration 0. These simulations are usually scattered over the entire height of the figure. You should only consider discarding samples if the samples are not distributed over the height of the figure. 

Options
-------
* ``discard_from_start``: Integer. Number of samples to discard from the start of the data. This is helpful for simulations where the minimized energy configuration throws off the equilibration detection.

In [ ]:
equilibration_figure = report.generate_equilibration_plots(discard_from_start=1)

#### --> Which process equilibrates more quickly, "complex" or "solvent"?

## Additional Decorrelation Analysis
==================

The following Pie Charts show you the breakdown of how many samples were kept, and how many were lost to either equilibration or decorrelation. Warnings are shown when below a threshold (originally written to be 10%)

In [ ]:
decorrelation_figure = report.generate_decorrelation_plots(decorrelation_threshold=0.1)

## Mixing statistics
=================

We can analyze the "mixing statistics" of the equilibrated part of the simulation to ensure that the $(X,S)$ chain is mixing reasonably well among the various alchemical states.

For information on how this is computed, including how to interpret the Perron Eigenvalue, please see the *Mixing Statistics Primer* at the end of the report.


What do you want to see?
-----------------------
You want a replica to mix into other replicas, so you want a diffusion of configurations shown by a spread out color map in the figure. What you don't want to see is highly concentrated replicas that do not mix at all. The graphs will show red and generate a warning if there are replicas that do not mix well.

For the Perron/subdominant eigenvalue, you want to see a value smaller than one `1`. The further away, the better. This number gives you an estimate of how many iterations it will take to equilibrate the current data. Keep in mind that this analysis only runs on the *already equilibrated data* and is therefor an estimate of how long it takes the system to relax in state and configuration space from this point.


Seeing something odd?
--------------------
* **The diagonal is very dark, but everything else is white**

You probably have poor mixing between states. This happens when there is insufficient phase space overlap between states and the probability of two replicas at different states swapping configurations approaches zero. If you have set the `mixing_warning_cutoff`, many of these states will be highlighted as warnings.

*Solution*: Add additional states to your simulation near the states which are not mixing well. Provide a more gradual change of energy from the state to improve replica exchange from that state.

* **Graph is mostly white!**

This can happen if you have _too_ good of mixing alongside too many states. In this case, mixing between all states is happening so regularly that there is no concentration of configurations in one state.

*Solution*: Reduce `mixing_cutoff`.

* **Its still way too white**

That is a limitation of the custom colormap. You can try un-commenting the line `cmap = plt.get_cmap("Blues")` below to get a blue-scale colormap which has a far smaller white level so you can better see the diffusion in blue. You will lose the red warning color of states with too low a swap rate, but you can always comment the line back out to see those. The warning message will still be generated.

*Solution*: Override the custom colormap that the function uses by setting `cmap_override="Blues"` or any other registered `matplotlib` colormap name.

Options
-------
You can adjust the `mixing_cutoff` options to control what threshold to display mixing. Anything below the cutoff will be shown as a blank. Defaults to `0.05`. Accepts either a float from `[0,1]` or `None` (`None` and `0` yield the same result)

The `mixing_warning_threshold` is the level at which you determine there were insufficient number of swaps between states. Consider adding additional states between the warnings and adjacent states to improve mixing. Accepts a float between `(mixing_cutoff,1]` (must be larger than `mixing_cutoff`). Defaults to `0.9` but this should be tuned based on the number of states.

In [ ]:
mixing_figure = report.generate_mixing_plot(mixing_cutoff=mixing_cutoff, 
                            mixing_warning_threshold=mixing_warning_threshold, 
                            cmap_override=None)

#### --> Could it help to add replicas to the complex or solvent phase? Which one and where?

## Replica Pseudorandom Walk Examination
====================

This section checks to see if all the replicas are exchanging states over the whole thermodynamic state space. This is different from tracking states as any replica is a continuous trajectory of configurations, just undergoing different forces at different times.

What do I want to see here?
-------------------------

Each plot is its own replica, the line in each plot shows which *state* a given replica is in at time. The ideal scenario is that all replicas visit all states numerous times. If you see a line that is relatively flat, then you can surmise that very little mixing is occurring from that replica and you may wish to consider adding more states around the stuck region to "un-stick" it.

Something seem odd?
------------------
* **All I see is black with some white dots mixed in (uncommon)**

This is a good thing! It means the replicas are well mixed and are rapidly changing states. There may be some phases which were redundant though, which is not necessarily a bad thing since it just adds more samples at the given state, but it may mean you did extra work. An example of this is *decoupling* the steric forces of a ligand once *electrostatics have been annihilated* in *implicit* solvent. Since there is no change to the intra-molecular interactions at this point and the most solvent models are based on partial charges (which are now 0), all changes to the sterics are the same state.

* **Some or All of my replicas stayed in the same state**

A sign of very poor mixing. Consider adding additional states (see the **Mixing Statistics** section above for ideas on where). There may be other factors such as a low number of attempted replica swaps between each iteration.


In [ ]:
replica_mixing_figure = report.generate_replica_mixing_plot(phase_stacked_replica_plots=phase_stacked_replica_plots)

#### --> Do any replicas remain in a single state across all iterations?

----
## Primers
====

### Equilibration Primer
===========

Is equilibration necessary?
---------------------------

In principle, we don't need to discard initial "unequilibrated" data; the estimate over a very long trajectory will converge to the correct free energy estimate no matter what---we simply need to run long enough.  Some MCMC practitioners, like Geyer, feel strongly enough about this to throw up a webpage in defense of this position:

http://users.stat.umn.edu/~geyer/mcmc/burn.html

In practice, if the initial conditions are very atypical of equilibrium (which is often the case in molecular simulation), it helps a great deal to discard an initial part of the simulation to equilibration.  But how much?  How do we decide?

Determining equilibration in a replica-exchange simulation
----------------------------------------------------------

For a standard molecular dynamics simulation producing a trajectory $x_t$, it's reasonably straightforward to decide approximately how much to discard if human intervention is allowed.  We simply look at some property $A_t = A(x_t)$ over the course of the simulation---ideally, a property that we know has some slow behavior that may affect the quantities we are interested in computing ($A(x)$ is a good choice if we're interested in the expectation $<A>$) and find the point where $A_t$ seems to have "settled in" to typical equilibrium behavior.

If we're interested in a free energy, which is computed from the potential energy differences, let's suppose the potential energy $U(x)$ may be a good quantity to examine.

But in a replica-exchange simulation, there are K replicas that execute nonphysical walks on many potential energy functions $U_k(x)$.  What quantity do we look at here?

Let's work by analogy.  In a single simulation, we would plot some quantity related to the potential energy $U(x)$, or its reduced version $u(x) = \beta U(x)$.  This is actually the negative logarithm of the probability density $\pi(x)$ sampled, up to an additive constant:

$$\pi(x) = Z^{-1} e^{-u(x)}$$
$$u(x) = -\ln \pi(x) + c$$

For a replica-exchange simulation, the sampler state is given by the pair $(X,S)$, where $X = \{x_1, x_2, \ldots, x_K \}$ are the replica configurations and $S = \{s_1, s_2, \ldots, s_K\}$ is the vector of state index permutations associated with the replicas.  The total probability sampled is

$$\Pi(X,S) = \prod_{k=1}^K \pi_{s_k}(x_k) = (Z_1 \cdots Z_K) \exp\left[-\sum_{k=1}^K u_{s_k}(x_k)\right] = Q^{-1} e^{-u_*(X)}$$

where the pseudoenergy $u_*(X)$ for the replica-exchange simulation is defined as

$$u_*(X) \equiv \sum_{k=1}^K u_{s_k}(x_k)$$

That is, $u_*(X)$ is the sum of the reduced potential energies of each replica configuration at the current thermodynamic state it is visiting.

### Mixing Statistics Primer
=============

How we compute the mixing ratios
------------------------------

In practice, this is done by recording the number of times a replica transitions from alchemical state $i$ to state $j$ in a single iteration.  Because the overall chain must obey detailed balance, we count each transition as contributing 0.5 counts toward the $i \rightarrow j$ direction and 0.5 toward the $j \rightarrow i$ direction.  This has the advantage of ensuring that the eigenvalues of the resulting transition matrix among alchemical states are purely real.

Interpreting the Perron (subdominant/second) Eigenvalue 
----------------------------------------------------

If the subdominant eigenvalue would have been unity, then the chain would be *decomposable*, meaning that it completely separated into two separate sets of alchemical states that did not mix.  This would have been an indication of poor phase space overlap between some alchemical states.

In practice, it's a great idea to monitor these statistics as the simulation is running, even if no data is discarded to equilibration at that point.  They give not only a good idea of whether sufficient mixing is occurring, but it provides a lower bound on the mixing time in configuration space.

If the configuration $x$ sampling is infinitely fast so that $x$ can be considered to be at equilibrium given the instantaneous permutation $S$ of alchemical state assignments, the subdominant eigenvalue $\lambda_2 \in [0,1]$ gives an estimate of the mixing time of the overall $(X,S)$ chain:
    
$$\tau = \frac{1}{1 - \lambda_2}$$

Now, in most cases, the configuration $x$ sampling is *not* infinitely fast, but at least we can use $\tau$ to get a very crude estimate of how quickly each replica relaxes in $(X,S)$ space.

# Part 3 - Analysis of YANK results

In [ ]:
free_energy_trace_figure = report.free_energy_trace(discard_from_start=1, n_trace=10)

#### --> Does the binding free energy estimate appear converged? If so, how can you tell?

In [ ]:
report.generate_free_energy()

## Free Energy Difference
============

The free energy difference is shown last as the quality of this estimate should be gauged with the earlier sections. Although MBAR provides an estimate of the free energy difference and its error, it is still only an estimate. You should consider if you have a sufficient number of decorrelated samples, sufficient mixing/phase space overlap between states, and sufficient replica random walk to gauge the quality of this estimate.

## Free Energy Trace for Equilibrium Stability

The free energy difference alone, even with all the additional information previously, may still be an underestimate of the true free energy. One way to check this is to drop samples from the start and end of the simulation, and re-run the free energy estimate. Ideally, you would want to see the forward and reverse analysis be roughly converged for when more than 80% of the samples are kept, divergence when only 10-30% of the samples are kept is expected behavior. 

**Important**: The 100% kept samples free energy WILL be different than the free energy difference above. The data analyzed here is not subsampled as this is an equlibrium test only. This is also only for *sampled* states where as the free energy difference from above includes the unsampled states.

See Klimovich, Shirts, and Mobley (J Comput Aided Mol Des., 29(5) https://dx.doi.org/10.1007%2Fs10822-015-9840-9) for more information on this analysis

## What do I want to see here?

There are three plots: one for each phase, and the combination. You want the two traces to be on top of each other for at least some of the larger kept samples. The horizontal band is the 2 standard deviations of the free energy estimate when all 100% of the samples are kept and can be used as reference as the esimtate diverges at smaller numbers of kept samples. Error bars are shown as 2 standard deviations

## Extracting and aligning the bound state

In [ ]:
complex_nc = f"{WORKING_DIR}/BRD-IR6-implicit-repex/experiments/complex.nc"
complex_prmtop = f"{WORKING_DIR}/BRD-IR6-implicit-repex/setup/systems/BRD-IR6/complex.prmtop"
complex_inpcrd = f"{WORKING_DIR}/BRD-IR6-implicit-repex/setup/systems/BRD-IR6/complex.inpcrd"

sel_str_align = 'protein and name CA'
sel_str_store = 'protein or resname UNL'
stride = 1

In [ ]:
import MDAnalysis as mda
from MDAnalysis.coordinates.memory import MemoryReader
from MDAnalysis.analysis import align

ref = mda.Universe(complex_prmtop, complex_inpcrd)
complex = mda.Universe(complex_prmtop, complex_inpcrd)
sel_protein = complex.select_atoms("protein")
sel_store = complex.select_atoms(sel_str_store)

print(f"Number of atoms: {sel_protein.n_atoms} in protein, " + \
      f"{sel_store.n_atoms} selected")

remarks = []
remarks.append('REMARK    <-- AMBER ATOM TYPES ')
for n in range(0,len(sel_store.types),20):
  remarks.append('REMARK    ' +  ' '.join([f'{t:2s}' for t in sel_store.types[n:n+20]]))
remarks.append('REMARK    AMBER ATOM TYPES -->')
remarks = '\n' + '\n'.join(remarks)
sel_store.write('bound.pdb', remarks=remarks)

In [ ]:
# Go frame-by-frame through the trajectory of the bound state, 
#  1. aligning the protein to the reference
#  2. writing the trajectory to another file

import netCDF4 as netcdf

# Load netCDF4 file from YANK output
nc = netcdf.Dataset(complex_nc)
niterations = nc.variables['positions'].shape[0]
nstates = nc.variables['states'].shape[1]
natoms = nc.variables['positions'].shape[2]
print(f"{niterations} iterations (will skip first), {nstates} states, {natoms} atoms")

# Determine replica indices that belong to the bound state, which is state 0
replica_indices = \
  [list(nc.variables['states'][iteration,:]).index(0) \
    for iteration in range(0,niterations,stride)]

# Align and store snapshots, skipping first frame
print('These are RMSDs before and after alignment:')
writer = mda.Writer('bound.dcd', sel_store.n_atoms)
for frame in range(1,len(replica_indices)):
  coords = nc.variables['positions'][frame*stride,replica_indices[frame],:,:]*10.0
  complex.load_new(coords, format=MemoryReader)
  print(align.alignto(complex, ref, select=sel_str_align))
  writer.write(sel_store)

## Visualizing the bound ensemble

Now we will visualize a series of configurations from the bound thermodynamic state. This code can be modified to visualize another state.

The bound ensemble from replica exchange is a superior basis for binding pose prediction than **molecular docking** and conventional **molecular dynamics**. It is superior to
* molecular docking because it accounts for entropy. A configuration with low entropy will not be sampled very often.
* molecular dynamics because it can sample multiple conformations more readily. Replica exchange enables the bound state to hop between energy minima.

In [ ]:
def visualize_mda_universe(u, \
    sel_string='not ((resname WAT) or (resname HOH))', 
    styles=[{"cartoon": {'color': 'spectrum'}}]):
  """
  Inputs: 
  u : mdanalysis universe
  sel_string : mdanalysis selection string for visible atoms
  style : py3Dmol style
  """
  # The number of frames in the simulation
  number_frames_analysis = len(u.trajectory)
  if number_frames_analysis > 10:
    stride_animation = number_frames_analysis/10
  else:
    stride_animation = 1

  # Deleting previously stored frames as PDBs and removing warnings
  import warnings
  warnings.filterwarnings('ignore')
  !rm traj/frame[0-9]?.pdb 2> /dev/null
  
    # Helper classes to read and get PDB fields
  class Atom(dict):
    def __init__(self, line):
      self["type"] = line[0:6].strip()
      self["idx"] = line[6:11].strip()
      self["name"] = line[12:16].strip()
      self["resname"] = line[17:20].strip()
      self["resid"] = int(int(line[22:26]))
      self["x"] = float(line[30:38])
      self["y"] = float(line[38:46])
      self["z"] = float(line[46:54])
      self["sym"] = line[76:78].strip()

    def __str__(self):
      line = list(" " * 80)
      line[0:6] = self["type"].ljust(6)
      line[6:11] = self["idx"].ljust(5)
      line[12:16] = self["name"].ljust(4)
      line[17:20] = self["resname"].ljust(3)
      line[22:26] = str(self["resid"]).ljust(4)
      line[30:38] = str(self["x"]).rjust(8)
      line[38:46] = str(self["y"]).rjust(8)
      line[46:54] = str(self["z"]).rjust(8)
      line[76:78] = self["sym"].rjust(2)
      return "".join(line) + "\n"
          
  class Molecule(list):
    def __init__(self, file):
      for line in file:
        if "ATOM" in line or "HETATM" in line:
          self.append(Atom(line))
              
      def __str__(self):
        outstr = ""
        for at in self:
          outstr += str(at)
        return outstr

  # Write out frames for animation
  protein = u.select_atoms(sel_string) 
  i = 0
  if not os.path.isdir('traj'):
      os.mkdir('traj')
  for ts in u.trajectory[0:len(u.trajectory):int(stride_animation)]: 
      if i > -1:
          with mda.Writer(os.path.join('traj',f'frame{i}.pdb'), protein.n_atoms) as W:
              W.write(protein)
      i = i + 1
  # Load frames as molecules (py3Dmol let us visualize a single "molecule" per frame)
  molecules = []
  for i in range(int(len(u.trajectory)/int(stride_animation))):
      with open(os.path.join('traj',f'frame{i}.pdb')) as ifile:
          molecules.append(Molecule(ifile))

  models = ""
  for i in range(len(molecules)):
    models += "MODEL " + str(i) + "\n"
    for j,mol in enumerate(molecules[i]):
      models += str(mol)
    models += "ENDMDL\n"

  # Animation
  view = py3Dmol.view(width=800, height=600)
  view.addModelsAsFrames(models)
  for i, at in enumerate(molecules[0]):
      view.setStyle({'model': -1, 'serial': i+1}, at.get("pymol", styles[0]))
      if len(styles)>1:
        for style in styles[1:]:
          view.addStyle({'model': -1, 'serial': i+1}, at.get("pymol", style))

  for i in range(len(molecules)):
    view.addLabel(i, {'position':{'x':0, 'y':0, 'z':0}, 'frame':i})

  view.zoomTo()
  view.animate({'loop': "forward", 'reps': 0})
  return view

In [ ]:
bound = mda.Universe('bound.pdb', 'bound.dcd')

import py3Dmol
view = visualize_mda_universe(bound, 
  sel_string = f"resname UNL or protein", \
  styles = [{'stick':{}}, {"cartoon": {'color': 'spectrum'}}])

view.show()

## Predicting the bound pose

A challenge with predicing the binding pose is that the bound ensemble contains many structures, but that every structure has the same statistical weight. Thus, we should group together similar structures and call them a conformation.

This is the procedure that I have implemented below:
* align every protein structure according to alpha carbons
* calculating a symmetry-corrected RMSD matrix of the ligand atoms
* selecting a representative structure based on the medoid - the point closest to all other points - of each cluster
* ranking poses based on the population of each cluster

In [ ]:
# This dcd file keeps the predicted poses
poses_dcd = 'poses.dcd'

# A string to select ligand atoms
sel_str_ligand = 'resname UNL and not name H*'

In [ ]:
# A class to perform RMSD calculations
import numpy as np
from munkres import Munkres
from MDAnalysis.analysis.base import AnalysisBase

class HungarianRMSD(AnalysisBase):
  """Hungarian symmetry-corrected root mean square deviation
  
  HungarianRMSD(atomgroup, ref_conf)
  
  Arguments
  ---------
  atomgroup : AtomGroup
    AtomGroup
  ref_conf : numpy.array (Nx3)
    refrence configuration or None to use AtomGroup configuration

  See http://dock.compbio.ucsf.edu/DOCK_6/dock6_manual.htm#LigandRMSD
  """
  def __init__(self, atomgroup, ref_conf=None, atom_types=None, **kwargs):
    """
    sel is an AtomGroup object MDAnalysis.core.groups
    ref_conf is the default reference configuration, an Nx3 numpy array
    atom_types is a list of strings
    """
    super(HungarianRMSD, self).__init__(atomgroup.universe.trajectory, **kwargs)
    
    self._ag = atomgroup
    if ref_conf is not None:
      self._ref_conf = ref_conf
    else:
      self._ref_conf = np.copy(self._ag.positions)
      
    if atom_types is not None:
      self._atom_types = atom_types
    else:
      self._atom_types = self._ag.types
        
    self.atom_sets_to_compare = []
    atom_indices = np.array(range(len(self._atom_types)))
    for t in set(self._atom_types):
      indices_t = (self._atom_types == t)
      self.atom_sets_to_compare.append((sum(indices_t), atom_indices[indices_t]))    
    
    self.munkres = Munkres()
    
  def set_ref_conf(self, ref_conf):
    """
    Sets a new reference configuration
    """
    self._ref_conf = ref_conf

  def _prepare(self):
    self.rmsds = []

  def _single_frame(self):
    ssd = 0.
    conf = self._ag.positions
    for (nelements, atom_set) in self.atom_sets_to_compare:
      if nelements == 1:
        j = atom_set[0]
        ssd += np.sum(np.square(conf[j, :] - self._ref_conf[j, :]))
      else:
        cost_matrix = np.array([[\
          np.sum(np.square(conf[atom_set[j],:]-self._ref_conf[atom_set[k],:])) \
            for j in range(nelements)] \
              for k in range(nelements)])
        path = self.munkres.compute(cost_matrix)
        ssd += np.sum([np.sum(np.square(\
          conf[atom_set[j],:]-self._ref_conf[atom_set[k],:])) for (j,k) in path])
    self.rmsds.append(np.sqrt(ssd / self._ag.n_atoms))

In [ ]:
# Load the complex
sel_ligand = bound.select_atoms(sel_str_ligand)

bound_pdb_reader = mda.coordinates.PDB.PDBReader('bound.pdb')
AMBER_atom_types = []
for line in bound_pdb_reader.remarks[1:-1]:
  AMBER_atom_types += line.split()
ligand_AMBER_atom_types = np.array(AMBER_atom_types)[sel_ligand.indices]

In [ ]:
# Calculates the RMSD matrix
calcHungarianRMSD = HungarianRMSD(sel_ligand, atom_types = ligand_AMBER_atom_types)

import os, pickle

if not os.path.isfile('rmsds.pkl'):
  rmsds = []
  bound.trajectory.rewind()
  for frame in range(bound.trajectory.n_frames):
    calcHungarianRMSD.set_ref_conf(bound.trajectory[frame].positions[sel_ligand.indices,:])
    calcHungarianRMSD.start = frame + 1
    calcHungarianRMSD.run()
    print(f'Calculated {len(calcHungarianRMSD.rmsds)} rmsds relative to frame {frame}')
    rmsds += calcHungarianRMSD.rmsds[frame + 1:]
    bound.trajectory.next()
  rmsds = np.array(rmsds)
  F = open('rmsds.pkl','wb')
  pickle.dump(rmsds, F)
  F.close()
else:
  F = open('rmsds.pkl','rb')
  rmsds = pickle.load(F)
  F.close()

In [ ]:
from scipy.spatial.distance import squareform
rmsds_sq = squareform(rmsds)

%matplotlib inline
import matplotlib.pyplot as plt

plt.imshow(rmsds_sq);
plt.title('Pairwise RMSDs');
plt.colorbar();

In [ ]:
import scipy.cluster
Z = scipy.cluster.hierarchy.linkage(rmsds, method='complete')
assignments = np.array(\
  scipy.cluster.hierarchy.fcluster(Z, 3, criterion='distance'))

# Reindexes the assignments in order of appearance
new_index = 0
mapping_to_new_index = {}
for assignment in assignments:
  if not assignment in mapping_to_new_index.keys():
    mapping_to_new_index[assignment] = new_index
    new_index += 1
assignments = [mapping_to_new_index[a] for a in assignments]

In [ ]:
plt.plot(assignments,'.-')
plt.title('Cluster assignments');

#### --> Interpret the cluster assignments. Are there clusters that reflect unequilibrated states? Are there recurrent clusters?

In [ ]:
# Create a list of how many frames are in a cluster and the representative frame from each cluster
counts_and_medoids = []
for n in range(max(assignments) + 1):
  inds = [i for i in range(len(assignments)) if assignments[i] == n]
  rmsds_n = rmsds_sq[inds][:, inds]
  counts_and_medoids.append((len(inds), inds[np.argmin(np.mean(rmsds_n, 0))]))
counts_and_medoids.sort(reverse=True)
print(counts_and_medoids)

In [ ]:
# Writes the representative frames to a new dcd file in decreasing order of population
from MDAnalysis.coordinates.memory import MemoryReader
writer = mda.Writer(poses_dcd, complex.atoms.n_atoms)
complex_out = mda.Universe('bound.pdb')

for (count, medoid) in counts_and_medoids:
  if count<20:
    continue
  positions = bound.trajectory[medoid].positions
  complex_out.load_new(positions, format=MemoryReader)
  writer.write(complex_out)

In [ ]:
import MDAnalysis as mda
complex = mda.Universe('bound.pdb', 'poses.dcd')

view = visualize_mda_universe(complex, 
  sel_string = f"resname UNL or protein", \
  styles = [{'stick':{}}, {"cartoon": {'color': 'spectrum'}}])

view.show()

#### --> Describe how the most highly populated poses differ from each other.